In [1]:
from tensorflow.keras import layers
import tensorflow.keras.backend as K

import tensorflow as tf

In [ ]:
input_shape = (32, 32, 3)
num_classes = 2

In [2]:
def densenet(input_shape, num_classes=1000, nb_dense_block=4, growth_rate=48, nb_filter=96, reduction=0.0, dropout_rate=0.0, weight_decay=1e-4, weights_path=None):
    eps = 1.1e-5

    # compute compression factor
    compression = 1.0 - reduction

    inputs = layers.Input(input_shape)

    nb_filter = 96
    nb_layers = [6,12,36,24]

    # Initial convolution
    net = layers.ZeroPadding2D((3, 3))(inputs)
    net = layers.Conv2D(nb_filter, (7, 7), strides=(2, 2), use_bias=False)(net)
    net = layers.BatchNormalization(epsilon=eps, axis=-1)(net)
    net = layers.Activation('relu')(net)
    net = layers.ZeroPadding2D((1, 1))(net)
    net = layers.MaxPooling2D((3, 3), strides=(2, 2))(net)

    for block_idx in range(nb_dense_block - 1):
        stage = block_idx+2
        net, nb_filter = dense_block(net, stage, nb_layers[block_idx], 
                                   nb_filter, growth_rate, 
                                   dropout_rate=dropout_rate, 
                                   weight_decay=weight_decay)

        net = transition_block(net, stage, nb_filter, 
                               compression=compression, 
                               dropout_rate=dropout_rate, 
                               weight_decay=weight_decay)
        nb_filter = int(nb_filter * compression)

    final_stage = stage + 1
    net, nb_filter = dense_block(net, final_stage, nb_layers[-1], 
                                 nb_filter, growth_rate, 
                                 dropout_rate=dropout_rate, 
                                 weight_decay=weight_decay)

    net = layers.BatchNormalization(epsilon=eps, axis=-1)(net)
    net = layers.Activation('relu')(net)
    net = layers.GlobalAveragePooling2D()(net)

    net = layers.Dense(num_classes)(net)
    net = layers.Activation('softmax')(net)

    model = tf.keras.Model(inputs, net, name='densenet')
    
    return model


def conv_block(net, stage, branch, nb_filter, dropout_rate=None, weight_decay=1e-4):
    eps = 1.1e-5
    
    inter_channel = nb_filter * 4  
    net = layers.BatchNormalization(epsilon=eps, axis=-1)(net)
    net = layers.Activation('relu')(net)
    net = layers.Conv2D(inter_channel, (1, 1), use_bias=False)(net)

    if dropout_rate:
        net = Dropout(dropout_rate)(net)

    net = layers.BatchNormalization(epsilon=eps, axis=-1)(net)
    net = layers.Activation('relu')(net)
    net = layers.ZeroPadding2D((1, 1))(net)
    net = layers.Conv2D(nb_filter, (3, 3), use_bias=False)(net)

    if dropout_rate:
        net = Dropout(dropout_rate)(net)

    return net


def transition_block(net, stage, nb_filter, compression=1.0, dropout_rate=None, weight_decay=1E-4):
    eps = 1.1e-5

    net = layers.BatchNormalization(epsilon=eps, axis=-1)(net)
    net = layers.Activation('relu')(net)
    net = layers.Conv2D(int(nb_filter * compression), (1, 1), use_bias=False)(net)

    if dropout_rate:
        net = Dropout(dropout_rate)(net)

    net = layers.AveragePooling2D((2, 2), strides=(2, 2))(net)

    return net


def dense_block(net, stage, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1e-4, grow_nb_filters=True):
    ''' Build a dense_block where the output of each conv_block is fed to subsequent ones
        # Arguments
            x: input tensor
            stage: index for dense block
            nb_layers: the number of layers of conv_block to append to the model.
            nb_filter: number of filters
            growth_rate: growth rate
            dropout_rate: dropout rate
            weight_decay: weight decay factor
            grow_nb_filters: flag to decide to allow number of filters to grow
    '''

    eps = 1.1e-5
    concat_feat = net

    for i in range(nb_layers):
        branch = i+1
        net = conv_block(concat_feat, stage, branch, growth_rate, dropout_rate, weight_decay)
        concat_feat = layers.concatenate([concat_feat, net], axis=-1)

        if grow_nb_filters:
            nb_filter += growth_rate

    return concat_feat, nb_filter

In [3]:
model = densenet(input_shape, num_classes)

Instructions for updating:
Colocations handled automatically by placer.
